In [41]:
import pandas as pd 
import numpy as np 
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

In [42]:
df = pd.read_csv('data/materias_enriquecida.csv')

In [43]:
vectorizer = CountVectorizer()

dfi = df.dropna(subset = ['artigo_ex'])

X = vectorizer.fit_transform(dfi['artigo_ex'])
y = dfi['assunto']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [44]:
# Treinamento do modelo Naive Bayes
modelo = MultinomialNB()
modelo.fit(X_train, y_train)

# Previsão
dfi['predict'] = modelo.predict(X)

dfi['flag_treino'] = np.where(dfi.index.isin(y_test.index), 0, 1)


c:\Users\55119\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\Users\55119\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [45]:
n_acertos_test = (dfi[dfi['flag_treino'] == 0]['assunto'] == dfi[dfi['flag_treino'] == 0]['predict']).sum()
n_test = dfi[dfi['flag_treino'] == 0]['assunto'].shape[0]
print(f'Percentagem de acertos: {round(n_acertos_test*100/n_test, 2)}%')

Percentagem de acertos: 96.44%


In [46]:
pd.pivot_table(dfi[dfi['flag_treino'] == 0],
                index = 'assunto',
                columns = 'predict',
                values = 'artigo_ex',
                aggfunc = len,
                margins = True
)

predict,economia,esportes,politica,All
assunto,,,,
economia,118.0,NaN,NaN,118
esportes,7.0,797.0,NaN,804
politica,27.0,2.0,61.0,90
All,152.0,799.0,61.0,1012


In [47]:
pd.pivot_table(dfi,
                index = 'assunto',
                columns = 'predict',
                values = 'artigo_ex',
                aggfunc = len,
                margins = True
)

predict,economia,esportes,politica,All
assunto,,,,
economia,473,1,1,475
esportes,12,3231,2,3245
politica,68,3,256,327
All,553,3235,259,4047


In [48]:
dfi['assunto'].value_counts()

esportes    3245
economia     475
politica     327
Name: assunto, dtype: int64